In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz
from clean_data import clean_results_df, clean_stats_df, update_player_listings, clean_rankings, clean_tournaments 
import numpy as np

## Clean DataFrames

In [2]:
df_top = pd.read_csv("s3://matchedge-pipeline/data/raw/top_500_players_23_June.csv")
df_all_tournament = pd.read_csv("s3://matchedge-pipeline/data/raw/All_Tournaments.csv", sep=',')
df_results = pd.read_csv('s3://matchedge-pipeline/data/raw/all_results.csv')

In [3]:
df_top = clean_rankings(df_top)
df_all_tournament = clean_tournaments(df_all_tournament)
df_results = clean_results_df(df_results)

Save as csv

In [5]:
df_top.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/top_500_players.csv', sep=",", columns=df_top.columns, index=False)
df_top.to_csv("s3://matchedge-pipeline/data/clean/top_500_players.csv", sep=",", columns=df_top.columns, index=False)


df_all_tournament.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/all_tournaments.csv', sep=',', columns=df_all_tournament.columns, index=False)
df_all_tournament.to_csv('s3://matchedge-pipeline/data/clean/all_tournaments.csv', sep=',', columns=df_all_tournament.columns, index=False)

df_results.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/all_results.csv', sep=',', columns=df_results.columns, index=False)
df_results.to_csv('s3://matchedge-pipeline/data/clean/all_results.csv', sep=',', columns=df_results.columns, index=False)

---
## Clean Stat Data

In [6]:
df_all_stats = pd.read_csv('s3://matchedge-pipeline/data/raw/all_stats_GS.csv', sep=',')

In [7]:
df_all_stats = clean_stats_df(df_all_stats)

Save as csv

In [23]:
df_all_stats.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/all_stats.csv', sep=',', columns=df_all_stats.columns, index=False)
df_all_stats.to_csv("s3://matchedge-pipeline/data/clean/all_stats.csv", sep=',', columns=df_all_stats.columns, index=False)

---
---
## Create First Player Listing (Do Only Once With Cleaning)

In [25]:
listing = update_player_listings()

In [32]:
listing.to_csv("s3://matchedge-pipeline/data/clean/player_archive.csv", columns=listing.columns, sep=',')
listing.to_csv("/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/player_archive.csv", columns=listing.columns, sep=',')

### Moving Forward Notes

Run ```update_player_listings()``` after scraping so can use ```player_archive.csv``` to clean results and stats. Call it with ```update_player_listings(df_live_rankings=df_top_scraped, df_results = df_results_scraped, df_stats = df_stats_scraped)```

---
---
## Combine Relevant Data Into One Table

In [26]:
results_414 = df_results[df_results['tournament_id']==414]
tournament_414 = df_all_tournament[df_all_tournament['id']==414]

other_results = df_results[df_results['tournament_id'] != 414]
other_tournaments = df_all_tournament[df_all_tournament['id'] != 414]

merged_tourn_results_414 = pd.merge(
    results_414, tournament_414[['id', 'level', 'location', 'surface', 'year']],
    left_on=['tournament_id', 'year'],
    right_on=['id', 'year'],
    how='left'
)

other_merged_results = pd.merge(
    other_results, other_tournaments[['id', 'level', 'location', 'surface']],
    left_on=["tournament_id"],
    right_on=["id"],
    how="left"
)

merged_tourn_results_414 = merged_tourn_results_414.drop(columns=["id"])
other_merged_results = other_merged_results.drop(columns=["id"])

merged_tourn_result_trial = pd.concat([merged_tourn_results_414, other_merged_results], ignore_index=True)

In [9]:
df_all_stats_trial = df_all_stats.copy()

---
---

#### Match p1,p1 in Stats and Results Tables

In [60]:
i = 0

p1_cols = ['player_1', 'player_1_scores', 'p1_id', 'p1_set1', 'p1_set2', 'p1_set3', 'p1_set4', 'p1_set5']
p2_cols = ['player_2', 'player_2_scores', 'p2_id', 'p2_set1', 'p2_set2', 'p2_set3', 'p2_set4', 'p2_set5']

valid_pairs1 = set(zip(df_all_stats_trial['tournament_id'], df_all_stats_trial['match_id']))
valid_pairs2 = set(zip(df_all_stats_trial['tournament_id'], df_all_stats_trial['match_id'], df_all_stats_trial['p1_id']))
valid_pairs3 = set(zip(df_all_stats_trial['tournament_id'], df_all_stats_trial['match_id'], df_all_stats_trial['p2_id']))
valid_pairs4 = set(zip(df_all_stats_trial['tournament_id'], df_all_stats_trial['p1_id'], df_all_stats_trial['p2_id']))


def reorder_players(row):
    global i
    tid, mid = row['tournament_id'], row['match_id']
    p1_id, p2_id = row['p1_id'], row['p2_id']
    
    orig_pair1 = (tid, mid)
    pair2_p1 = (tid, mid, p1_id)
    pair2_p2 = (tid, mid, p2_id)
    pair_no_match = (tid, p1_id, p2_id)
    swapped_no_match = (tid, p2_id, p1_id)
    
    ########################################################################################################################
    # Special case for tournament 414
    if tid == 414:
        if pair_no_match in valid_pairs4:
            return row
        elif swapped_no_match in valid_pairs4:
            for c1, c2 in zip(p1_cols, p2_cols):
                row[c1], row[c2] = row[c2], row[c1]
            return row
        else:
            print(f"[FAIL: 414 special case] tournament_id=414, p1_id={p1_id}, p2_id={p2_id}")
            return row
    ########################################################################################################################

    if pd.notna(mid) and orig_pair1 in valid_pairs1:
        if pair2_p1 in valid_pairs2 or pair2_p2 in valid_pairs3:
            return row
        elif pair2_p1 in valid_pairs3 or pair2_p2 in valid_pairs2:
            for c1, c2 in zip(p1_cols, p2_cols):
                row[c1], row[c2] = row[c2], row[c1]
            return row
        else:
            print(f"[FAIL: valid match_id, but player ID mismatch] match_id={mid}, tournament_id={tid}, p1_id={p1_id}, p2_id={p2_id}")
            return row
    elif pd.isna(mid):
        if pair_no_match in valid_pairs4:
            return row
        elif swapped_no_match in valid_pairs4:
            for c1, c2 in zip(p1_cols, p2_cols):
                row[c1], row[c2] = row[c2], row[c1]
            return row
        else:
            print(f"[FAIL: missing match_id, and player ID pair not found] tournament_id={tid}, p1_id={p1_id}, p2_id={p2_id}")
            return row
    else:
        print(f"[FAIL: match_id not in valid_pairs1] match_id={mid}, tournament_id={tid}")
        return row


In [61]:
merged_tourn_result_trial = merged_tourn_result_trial.apply(reorder_players, axis=1)


[FAIL: 414 special case] tournament_id=414, p1_id=m0gz, p2_id=bk92
[FAIL: 414 special case] tournament_id=414, p1_id=sg56, p2_id=<NA>
[FAIL: 414 special case] tournament_id=414, p1_id=f0f1, p2_id=r0dg
[FAIL: 414 special case] tournament_id=414, p1_id=ta29, p2_id=o513
[FAIL: 414 special case] tournament_id=414, p1_id=d0c1, p2_id=<NA>
[FAIL: 414 special case] tournament_id=414, p1_id=mw75, p2_id=ss25
[FAIL: match_id not in valid_pairs1] match_id=qs024, tournament_id=339
[FAIL: match_id not in valid_pairs1] match_id=qs025, tournament_id=339
[FAIL: match_id not in valid_pairs1] match_id=qs027, tournament_id=339
[FAIL: match_id not in valid_pairs1] match_id=ms023, tournament_id=8998
[FAIL: match_id not in valid_pairs1] match_id=ms023, tournament_id=301
[FAIL: match_id not in valid_pairs1] match_id=ms008, tournament_id=580
[FAIL: match_id not in valid_pairs1] match_id=ms009, tournament_id=580
[FAIL: match_id not in valid_pairs1] match_id=ms010, tournament_id=580
[FAIL: match_id not in valid_

In [ ]:
df_all_stats_trial[df_all_stats_trial['tournament_id']==414]['p1_id']

1391    False
1392    False
1393    False
1394    False
1395    False
        ...  
1655    False
1656    False
1657    False
1658    False
1659    False
Name: p1_id, Length: 82, dtype: bool

In [46]:
merged_tourn_result_trial[merged_tourn_result_trial['tournament_id']==414]['p1_id'].isna()

0     False
1     False
2     False
3     False
4     False
      ...  
81    False
82    False
83    False
84    False
85    False
Name: p1_id, Length: 86, dtype: bool

In [74]:
merge_keys = ['tournament_id', 'match_id', 'player_1', 'player_2']

merged_tourn_results_414 = merged_tourn_result_trial[merged_tourn_result_trial['tournament_id']==414]
stat_414 = df_all_stats_trial[df_all_stats_trial['tournament_id']==414]

merged_stats_414 = pd.merge(
    merged_tourn_results_414,
    stat_414, on=merge_keys,
    how='left'
)

# Drop all _y columns
merged_stats_414 = merged_stats_414.loc[:, ~merged_stats_414.columns.str.endswith('_y')]

# Rename _x columns (remove suffix)
merged_stats_414.columns = merged_stats_414.columns.str.replace('_x$', '', regex=True)

In [75]:
merge_keys = ['tournament_id', 'match_id']

merged_tourn_results_not_414 = merged_tourn_result_trial[merged_tourn_result_trial['tournament_id']!=414]
stat_not_414 = df_all_stats_trial[df_all_stats_trial['tournament_id']!=414]

merged_stats_not_414 = pd.merge(
    merged_tourn_results_not_414,
    stat_not_414, on=merge_keys,
    how='left'
)

# Drop all _y columns
merged_stats_not_414 = merged_stats_not_414.loc[:, ~merged_stats_not_414.columns.str.endswith('_y')]

# Rename _x columns (remove suffix)
merged_stats_not_414.columns = merged_stats_not_414.columns.str.replace('_x$', '', regex=True)

In [73]:
print(merged_stats_not_414.columns == merged_stats_414.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [82]:
merged_tourn_stat_trial = pd.concat([merged_stats_414, merged_stats_not_414], ignore_index=True)

In [83]:
cols = [
    "p1_first_serve", "p1_1st_serve_points_won", "p1_2nd_serve_points_won",
    "p1_1st_serve_return_points_won", "p1_service_points_won", "p1_2nd_serve_return_points_won",
    "p1_return_points_won", "p1_total_points_won",
    "p2_first_serve", "p2_1st_serve_points_won", "p2_2nd_serve_points_won",
    "p2_1st_serve_return_points_won", "p2_service_points_won", "p2_2nd_serve_return_points_won",
    "p2_return_points_won", "p2_total_points_won"
]

merged_tourn_stat_trial = merged_tourn_stat_trial.dropna(subset=cols, how='all')


In [84]:
merged_tourn_stat_trial

,match_date,player_1,player_2,duration,match_round,player_1_scores,player_2_scores,winner,result,match_id,...,p2_service_points_won,p2_return_points_won,p2_total_points_won,p2_max_speed,p2_1st_serve_average_speed,p2_2nd_serve_average_speed,p2_break_point_opportunities,p1_break_point_opportunities,p1_net_points_played,p2_net_points_played
0,2025-05-24,A. Rublev,F. Cobolli,0 days 01:28:37,Final,2 4,6 6,Flavio Cobolli,Completed,ms001,...,0.61,0.50,0.57,NaN,NaN,NaN,4.0,7.0,7.0,5.0
1,2025-05-23,F. Auger-Aliassime,A. Rublev,0 days 01:25:13,Semi-Finals,1 4,6 6,Andrey Rublev,Completed,ms002,...,0.73,0.43,0.58,NaN,NaN,NaN,7.0,1.0,2.0,8.0
2,2025-05-23,T. Etcheverry,F. Cobolli,0 days 02:47:54,Semi-Finals,6 5 4,2 7 6,Flavio Cobolli,Completed,ms003,...,0.60,0.38,0.49,NaN,NaN,NaN,10.0,7.0,16.0,10.0
3,2025-05-22,A. Rublev,L. Darderi,0 days 01:53:35,Quarter-Finals,6 3 6,1 6 3,Andrey Rublev,Completed,ms005,...,0.61,0.31,0.46,NaN,NaN,NaN,6.0,7.0,12.0,4.0
4,2025-05-22,A. Muller,F. Auger-Aliassime,0 days 02:58:31,Quarter-Finals,6(4) 7 3,7 6(6) 6,Felix Auger-Aliassime,Completed,ms004,...,0.64,0.38,0.51,NaN,NaN,NaN,12.0,6.0,15.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3833,2024-12-22,J. Shang,N. Basavareddy,0 days 01:33:00,Round Robin -,4 2 2 1,3(4) 4 4 4,Nishesh Basavareddy,Completed,ms020,...,0.69,0.42,0.54,NaN,NaN,NaN,11.0,6.0,4.0,9.0
3834,2024-12-22,A. Fils,J. Fonseca,0 days 01:50:00,Round Robin -,4 2 1 4 1,3(9) 4 4 1 4,Joao Fonseca,Completed,ms010,...,0.70,0.33,0.52,NaN,NaN,NaN,11.0,5.0,8.0,9.0
3835,2024-12-22,J. Mensik,L. Tien,0 days 02:17:00,Round Robin -,3(6) 3(3) 4 4 3(8),4 4 2 2 4,Learner Tien,Completed,ms011,...,0.64,0.26,0.47,NaN,NaN,NaN,1.0,9.0,22.0,9.0
3836,2024-12-22,A. Michelsen,N. Basavareddy,0 days 01:53:00,Round Robin -,2 4 4 4,4 3(5) 3(4) 2,Alex Michelsen,Completed,ms018,...,0.66,0.35,0.50,NaN,NaN,NaN,5.0,7.0,20.0,16.0


This is good! there are only 4 missing rows that are available in ```df_all_stats_trial```.

---
---


In [85]:
merged_tourn_stat_trial.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3704 entries, 0 to 3837
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype          
---  ------                          --------------  -----          
 0   match_date                      3704 non-null   datetime64[ns] 
 1   player_1                        3704 non-null   object         
 2   player_2                        3704 non-null   object         
 3   duration                        3242 non-null   timedelta64[ns]
 4   match_round                     3704 non-null   object         
 5   player_1_scores                 3704 non-null   object         
 6   player_2_scores                 3704 non-null   object         
 7   winner                          3704 non-null   object         
 8   result                          3704 non-null   object         
 9   match_id                        3704 non-null   object         
 10  tournament_id                   3704 non-null   int64          
 

In [86]:
merged_tourn_stat_trial[merged_tourn_stat_trial['p1_id'].isna()]

,match_date,player_1,player_2,duration,match_round,player_1_scores,player_2_scores,winner,result,match_id,...,p2_service_points_won,p2_return_points_won,p2_total_points_won,p2_max_speed,p2_1st_serve_average_speed,p2_2nd_serve_average_speed,p2_break_point_opportunities,p1_break_point_opportunities,p1_net_points_played,p2_net_points_played
250,2025-01-05,A. Klintcharov,S. Nagal,0 days 01:54:40,1st Round Qualifying,6 3 1,1 6 6,Sumit Nagal,Completed,qs009,...,0.67,0.43,0.54,NaN,NaN,NaN,6.0,5.0,19.0,18.0
446,2025-01-07,C. Hewitt,N. Basilashvili,NaT,1st Round Qualifying,1 4,6 6,Nikoloz Basilashvili,Completed,qs082,...,0.69,0.49,0.58,NaN,NaN,NaN,12.0,3.0,0.0,0.0
473,2025-01-06,T. van Rijthoven,J. Kym,NaT,1st Round Qualifying,0 3,6 6,Jerome Kym,Completed,qs115,...,0.76,0.54,0.66,NaN,NaN,NaN,7.0,3.0,0.0,0.0
614,2025-02-01,N. Visker,A. Vavassori,0 days 01:22:04,1st Round Qualifying,6(2) 2,7 6,Andrea Vavassori,Completed,qs008,...,0.80,0.42,0.58,NaN,NaN,NaN,5.0,3.0,0.0,0.0
811,2025-02-15,J. Lucas Reis Da Silva,C. Tseng,0 days 02:43:14,1st Round Qualifying,7 3 4,6(0) 6 6,Chun-Hsin Tseng,Completed,qs011,...,0.62,0.40,0.51,NaN,NaN,NaN,6.0,7.0,9.0,10.0
894,2025-02-22,R. De Alba,G. Diallo,0 days 01:13:07,1st Round Qualifying,1 1,6 6,Gabriel Diallo,Completed,qs009,...,0.76,0.57,0.65,NaN,NaN,NaN,11.0,0.0,4.0,9.0
933,2025-02-22,G. Alberto Olivieri,G. Heide,0 days 01:12:40,1st Round Qualifying,2 2,6 6,Gustavo Heide,Completed,qs011,...,0.79,0.50,0.63,NaN,NaN,NaN,10.0,1.0,4.0,11.0
934,2025-02-22,A. Daniel Vallejo,J. Ficovich,0 days 01:24:05,1st Round Qualifying,2 3,6 6,Juan Pablo Ficovich,Completed,qs013,...,0.71,0.49,0.59,NaN,NaN,NaN,13.0,4.0,10.0,4.0
1066,2025-03-03,T. Papamalamis,M. Kukushkin,0 days 03:26:16,1st Round Qualifying,4 7 5,6 6(4) 7,Mikhail Kukushkin,Completed,qs051,...,0.57,0.42,0.51,NaN,NaN,NaN,8.0,13.0,27.0,29.0
1310,2025-03-30,S. Adrian Andreescu,R. Albot,0 days 02:07:01,1st Round Qualifying,6 3 2,4 6 6,Radu Albot,Completed,qs015,...,0.54,0.52,0.53,NaN,NaN,NaN,17.0,14.0,9.0,12.0


In [88]:
merged_tourn_stat_trial[(merged_tourn_stat_trial['p1_id'].isna()) | merged_tourn_stat_trial['p2_id'].isna()]

,match_date,player_1,player_2,duration,match_round,player_1_scores,player_2_scores,winner,result,match_id,...,p2_service_points_won,p2_return_points_won,p2_total_points_won,p2_max_speed,p2_1st_serve_average_speed,p2_2nd_serve_average_speed,p2_break_point_opportunities,p1_break_point_opportunities,p1_net_points_played,p2_net_points_played
167,2024-12-29,G. Diallo,K. Lung Ng,0 days 01:51:53,1st Round Qualifying,7 6,6(2) 2,Gabriel Diallo,Completed,qs010,...,0.55,0.25,0.43,NaN,NaN,NaN,0.0,7.0,12.0,9.0
250,2025-01-05,A. Klintcharov,S. Nagal,0 days 01:54:40,1st Round Qualifying,6 3 1,1 6 6,Sumit Nagal,Completed,qs009,...,0.67,0.43,0.54,NaN,NaN,NaN,6.0,5.0,19.0,18.0
425,2025-01-07,Y. Uchiyama,A. Daniel Vallejo,NaT,2nd Round Qualifying,4 6 6,6 3 4,Yasutaka Uchiyama,Completed,qs052,...,0.59,0.38,0.48,NaN,NaN,NaN,9.0,8.0,0.0,0.0
446,2025-01-07,C. Hewitt,N. Basilashvili,NaT,1st Round Qualifying,1 4,6 6,Nikoloz Basilashvili,Completed,qs082,...,0.69,0.49,0.58,NaN,NaN,NaN,12.0,3.0,0.0,0.0
473,2025-01-06,T. van Rijthoven,J. Kym,NaT,1st Round Qualifying,0 3,6 6,Jerome Kym,Completed,qs115,...,0.76,0.54,0.66,NaN,NaN,NaN,7.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569,2024-10-20,R. Ymer,K. Majchrzak,0 days 00:49:00,1st Round Qualifying,1 1,6 6,Kamil Majchrzak,Completed,qs009,...,0.97,0.55,0.72,NaN,NaN,NaN,7.0,0.0,12.0,5.0
3571,2024-10-20,A. Reymond,H. Medjedovic,0 days 01:35:00,1st Round Qualifying,6(4) 5,7 7,Hamad Medjedovic,Completed,qs013,...,0.67,0.43,0.54,NaN,NaN,NaN,11.0,5.0,21.0,19.0
3769,2024-11-09,M. Maksimovic,T. Tirante,0 days 01:47:00,1st Round Qualifying,6(7) 3,7 6,Thiago Agustin Tirante,Completed,qs011,...,0.67,0.42,0.53,NaN,NaN,NaN,7.0,3.0,0.0,0.0
3794,2024-11-09,Q. Halys,T. Papamalamis,0 days 01:04:00,Round of 32,6 6,3 1,Quentin Halys,Completed,ms029,...,0.47,0.14,0.34,NaN,NaN,NaN,0.0,10.0,15.0,1.0


In [99]:
merged_tourn_stat_trial.to_csv('/Users/samueleferrucci/Documents/Coding/Projects/Tennis ML/data/clean/merged_matches.csv', sep=',', columns=merged_tourn_stat_trial.columns, index=False)
merged_tourn_stat_trial.to_csv('s3://matchedge-pipeline/data/clean/merged_matches.csv', sep=',', columns=merged_tourn_stat_trial.columns, index=False)